In [1]:
import os

import flwr as fl
import tensorflow as tf

2023-07-19 14:12:52.622757: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 14:12:52.749096: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-19 14:12:53.169448: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-19 14:12:53.169519: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Load model and data (MobileNetV2, CIFAR-10)
model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

2023-07-19 14:12:53.753098: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-19 14:12:53.779069: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-07-19 14:12:53.779087: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-07-19 14:12:53.779543: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (o

In [3]:
# Define Flower client
class CifarClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        #print(f"[Client {self}] config")
        #print(model.get_weights())
        print("testeeee")
        
        return model.get_weights()

    def fit(self, parameters, config):
        model.set_weights(parameters)
        model.fit(x_train, y_train, epochs=1, batch_size=32)
        return model.get_weights(), len(x_train), {}

    def evaluate(self, parameters, config):
        """Evaluate parameters on the locally held test set."""

        # Update local model with global parameters
        
        model.set_weights(parameters)
        # Evaluate global model parameters on the local test data
        loss, accuracy = model.evaluate(x_test, y_test)
        #loss, accuracy = model.evaluate(x_test, y_test)
        print("test metrics: ",accuracy)
        
        # Return results, including the custom accuracy metric
        num_examples_test = len(x_test)
        return loss, num_examples_test, {"accuracy": accuracy}

In [5]:
# Start Flower client
fl.client.start_numpy_client(server_address="127.0.0.1:8080", client=CifarClient())

INFO flwr 2023-07-19 14:15:23,606 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-07-19 14:15:23,617 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2023-07-19 14:15:23,618 | connection.py:39 | ChannelConnectivity.CONNECTING
DEBUG flwr 2023-07-19 14:15:23,620 | connection.py:39 | ChannelConnectivity.READY
DEBUG flwr 2023-07-19 14:39:01,461 | connection.py:113 | gRPC channel closed

KeyboardInterrupt

